# 물체 검출 결과 평가 방법 이해하기
성능 평가 코드를 이해하고 분석하여 precision-recall curve에서 주어진 recall값에 대한 threshold 값을 계산합니다.<br/>
구해진 threshold값을 이용하여 검출 결과를 출력합니다.

In [1]:
"""Adapted from:
    @longcw faster_rcnn_pytorch: https://github.com/longcw/faster_rcnn_pytorch
    @rbgirshick py-faster-rcnn https://github.com/rbgirshick/py-faster-rcnn
    Licensed under The MIT License [see LICENSE for details]
"""

from __future__ import print_function
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from data import VOC_ROOT, VOCAnnotationTransform, VOCDetection, BaseTransform
from data import VOC_CLASSES as labelmap
import torch.utils.data as data
from utils import Timer
from ssd import build_ssd

import sys
import os
import time
import argparse
import numpy as np
import pickle
import cv2

# %matplotlib inline
from matplotlib import pyplot as plt

import xml.etree.ElementTree as ET

annopath = os.path.join(VOC_ROOT, 'VOC2007', 'Annotations', '%s.xml')
imgpath = os.path.join(VOC_ROOT, 'VOC2007', 'JPEGImages', '%s.jpg')
imgsetpath = os.path.join(VOC_ROOT, 'VOC2007', 'ImageSets', 'Main', '{:s}.txt')
YEAR = '2007'
devkit_path = VOC_ROOT + 'VOC' + YEAR
dataset_mean = (104, 117, 123)

In [2]:
### load net
num_classes = len(labelmap) + 1                      # +1 for background
net = build_ssd('vgg16', 300, 21)    # initialize SSD
# net.load_state_dict(torch.load(args.trained_model))
net.load_weights('weights/ssd300_mAP_77.43_v2.pth')
# net.load_state_dict(torch.load(args.trained_model, map_location='cpu'))
net.eval()
print('Finished loading model!')

### load data
set_type = 'test'
dataset = VOCDetection(VOC_ROOT, [('2007', set_type)],
                       BaseTransform(300, dataset_mean),
                       VOCAnnotationTransform())
if torch.cuda.is_available():
    net = net.cuda()
    cudnn.benchmark = True

Loading weights into state dict...
Finished!
Finished loading model!


In [8]:
save_folder = 'ssd300_results'
top_k = 10

from layers import Detect

num_images = len(dataset)
parser = Detect(num_classes, 0, 200, 0.01, 0.45)
softmax = nn.Softmax(dim=-1)

# all detections are collected into:
#    all_boxes[cls][image] = N x 5 array of detections in
#    (x1, y1, x2, y2, score)
all_boxes = [[[] for _ in range(num_images)]
             for _ in range(len(labelmap)+1)]

# timers
_t = {'im_detect': Timer(), 'misc': Timer()}    
        
output_dir = os.path.join(save_folder, set_type)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
det_file = os.path.join(output_dir, 'detections.pkl')

for i in range(num_images):
    im, gt, h, w, ids = dataset.pull_item_with_index(i)

    x = im.unsqueeze(0)
    if torch.cuda.is_available():
        x = x.cuda()
        
    _t['im_detect'].tic()
    with torch.no_grad():    
        loc_pred, cls_pred, priors = net(x)    
    detections = parser(loc_pred, softmax(cls_pred), priors.type(type(x)))                
    detect_time = _t['im_detect'].toc(average=False)    
    
    # skip j = 0, because it's the background class
    for j in range(1, detections.size(1)):
        dets = detections[0, j, :]
        mask = dets[:, 0].gt(0.).expand(5, dets.size(0)).t()
        dets = torch.masked_select(dets, mask).view(-1, 5)
        if dets.shape[0] == 0:
            continue
        boxes = dets[:, 1:]
        boxes[:, 0] *= w
        boxes[:, 2] *= w
        boxes[:, 1] *= h
        boxes[:, 3] *= h
        scores = dets[:, 0].cpu().numpy()
                
        ids_dets = ids*np.ones( (len(scores),1) )
        cls_dets = np.hstack(
            (boxes.cpu().numpy(),\
             scores[:, np.newaxis], \
             ids_dets
            )).astype(np.float32, copy=False)
        all_boxes[j][i] = cls_dets
        
    print('im_detect: {:d}/{:d} {:.3f}s'.format(i+1, num_images, detect_time))

with open(det_file, 'wb') as f:
    pickle.dump(all_boxes, f, pickle.HIGHEST_PROTOCOL)    

im_detect: 1/4952 2.237s
im_detect: 2/4952 2.140s
im_detect: 3/4952 2.138s


KeyboardInterrupt: 

In [6]:
def parse_rec(filename):
    """ Parse a PASCAL VOC xml file """
    tree = ET.parse(filename)
    objects = []
    for obj in tree.findall('object'):
        obj_struct = {}
        obj_struct['name'] = obj.find('name').text
        obj_struct['pose'] = obj.find('pose').text
        obj_struct['truncated'] = int(obj.find('truncated').text)
        obj_struct['difficult'] = int(obj.find('difficult').text)
        bbox = obj.find('bndbox')
        obj_struct['bbox'] = [int(bbox.find('xmin').text) - 1,
                              int(bbox.find('ymin').text) - 1,
                              int(bbox.find('xmax').text) - 1,
                              int(bbox.find('ymax').text) - 1]
        objects.append(obj_struct)

    return objects

In [9]:
def voc_eval(det_boxes, annopath, imagesetfile, classname, cachedir,
             ovthresh=0.5, use_07_metric=True):
    """rec, prec, ap = voc_eval(detpath,
                           annopath,
                           imagesetfile,
                           classname,
                           [ovthresh],
                           [use_07_metric])
Top level function that does the PASCAL VOC evaluation.
detpath: Path to detections
   detpath.format(classname) should produce the detection results file.
annopath: Path to annotations
   annopath.format(imagename) should be the xml annotations file.
imagesetfile: Text file containing the list of images, one image per line.
classname: Category name (duh)
cachedir: Directory for caching the annotations
[ovthresh]: Overlap threshold (default = 0.5)
[use_07_metric]: Whether to use VOC07's 11 point AP computation
   (default True)
"""
# assumes detections are in detpath.format(classname)
# assumes annotations are in annopath.format(imagename)
# assumes imagesetfile is a text file with each line an image name
# cachedir caches the annotations in a pickle file
# first load gt
    if not os.path.isdir(cachedir):
        os.mkdir(cachedir)
    cachefile = os.path.join(cachedir, 'annots.pkl')
    # read list of images
    with open(imagesetfile, 'r') as f:
        lines = f.readlines()
    imagenames = [x.strip() for x in lines]
    if not os.path.isfile(cachefile):
        # load annots
        recs = {}
        for i, imagename in enumerate(imagenames):
            recs[imagename] = parse_rec(annopath % (imagename))
            if i % 100 == 0:
                print('Reading annotation for {:d}/{:d}'.format(
                   i + 1, len(imagenames)))
        # save
        print('Saving cached annotations to {:s}'.format(cachefile))
        with open(cachefile, 'wb') as f:
            pickle.dump(recs, f)
    else:
        # load
        with open(cachefile, 'rb') as f:
            recs = pickle.load(f)

    # extract gt objects for this class
    class_recs = {}
    npos = 0
    for imagename in imagenames:
        R = [obj for obj in recs[imagename] if obj['name'] == classname]
        bbox = np.array([x['bbox'] for x in R])
        difficult = np.array([x['difficult'] for x in R]).astype(np.bool)
        det = [False] * len(R)
        npos = npos + sum(~difficult)
        class_recs[imagename] = {'bbox': bbox,
                                 'difficult': difficult,
                                 'det': det}

    # read dets
#     detfile = detpath.format(classname)
#     with open(detfile, 'r') as f:
#         lines = f.readlines()

    if len(det_boxes):
        BB = det_boxes[:,:4]
        confidence = det_boxes[:,4]
        image_ids = det_boxes[:,-1]        

        # sort by confidence
        sorted_ind = np.argsort(-confidence)
        sorted_scores = np.sort(-confidence)
        BB = BB[sorted_ind, :]
        image_ids = [image_ids[x] for x in sorted_ind]

        # go down dets and mark TPs and FPs
        nd = len(image_ids)
        tp = np.zeros(nd)
        fp = np.zeros(nd)
        for d in range(nd):
            R = class_recs[image_ids[d]]
            bb = BB[d, :].astype(float)
            ovmax = -np.inf
            BBGT = R['bbox'].astype(float)
            if BBGT.size > 0:
                # compute overlaps
                # intersection
                ixmin = np.maximum(BBGT[:, 0], bb[0])
                iymin = np.maximum(BBGT[:, 1], bb[1])
                ixmax = np.minimum(BBGT[:, 2], bb[2])
                iymax = np.minimum(BBGT[:, 3], bb[3])
                iw = np.maximum(ixmax - ixmin, 0.)
                ih = np.maximum(iymax - iymin, 0.)
                inters = iw * ih
                uni = ((bb[2] - bb[0]) * (bb[3] - bb[1]) +
                       (BBGT[:, 2] - BBGT[:, 0]) *
                       (BBGT[:, 3] - BBGT[:, 1]) - inters)
                overlaps = inters / uni
                ovmax = np.max(overlaps)
                jmax = np.argmax(overlaps)

            if ovmax > ovthresh:
                if not R['difficult'][jmax]:
                    if not R['det'][jmax]:
                        tp[d] = 1.
                        R['det'][jmax] = 1
                    else:
                        fp[d] = 1.
            else:
                fp[d] = 1.

        # compute precision recall
        fp = np.cumsum(fp)
        tp = np.cumsum(tp)
        rec = tp / float(npos)
        # avoid divide by zero in case the first detection matches a difficult
        # ground truth
        prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
        ap = voc_ap(rec, prec, use_07_metric)
    else:
        rec = -1.
        prec = -1.
        ap = -1.

    return rec, prec, ap


In [ ]:
print('Evaluating detections')

cachedir = os.path.join(save_folder, 'annotations_cache')
aps = []
# The PASCAL VOC metric changed in 2010
use_07_metric = True
print('VOC07 metric? ' + ('Yes' if use_07_metric else 'No'))

for i, cls in enumerate(labelmap):    
    
    import pdb
    pdb.set_trace()
    
    rec, prec, ap = voc_eval(
       all_boxes[i], annopath, imgsetpath.format(set_type), cls, cachedir,
       ovthresh=0.5, use_07_metric=use_07_metric)
    aps += [ap]
    print('AP for {} = {:.4f}'.format(cls, ap))
    with open(os.path.join(output_dir, cls + '_pr.pkl'), 'wb') as f:
        pickle.dump({'rec': rec, 'prec': prec, 'ap': ap}, f)
print('Mean AP = {:.4f}'.format(np.mean(aps)))
print('~~~~~~~~')
print('Results:')
for ap in aps:
    print('{:.3f}'.format(ap))
print('{:.3f}'.format(np.mean(aps)))
print('~~~~~~~~')
print('')
print('--------------------------------------------------------------')
print('Results computed with the **unofficial** Python eval code.')
print('Results should be very close to the official MATLAB eval code.')
print('--------------------------------------------------------------')
    
    
    
    
#     from data import VOC_CLASSES as labels
    

#     im = cv2.imread(dataset._imgpath % dataset.ids[i])

#     plt.figure(figsize=(10,10))
#     colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()
#     plt.imshow(im)  # plot the image for matplotlib
#     currentAxis = plt.gca()

#     import pdb
#     pdb.set_trace()
    
#     detections = cls_dets.copy()
#     # scale each detection back up to the image
#     scale = torch.Tensor(im.shape[1::-1]).repeat(2)
#     for i in range(detections.size(1)):
#         j = 0
#         while detections[0,i,j,0] >= 0.6:
#             score = detections[0,i,j,0]
#             label_name = labels[i-1]
#             display_txt = '%s: %.2f'%(label_name, score)
#             pt = (detections[0,i,j,1:]*scale).cpu().numpy()
#             coords = (pt[0], pt[1]), pt[2]-pt[0]+1, pt[3]-pt[1]+1
#             color = colors[i]
#             currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor=color, linewidth=2))
#             currentAxis.text(pt[0], pt[1], display_txt, bbox={'facecolor':color, 'alpha':0.5})
#             j+=1

#     print('im_detect: {:d}/{:d} {:.3f}s'.format(i + 1,
#                                                 num_images, detect_time))

# with open(det_file, 'wb') as f:
#     pickle.dump(all_boxes, f, pickle.HIGHEST_PROTOCOL)

# print('Evaluating detections')
# evaluate_detections(all_boxes, output_dir, dataset)

Evaluating detections
VOC07 metric? Yes
> <ipython-input-11-f091404ba971>(14)<module>()
-> rec, prec, ap = voc_eval(


(Pdb)  imgsetpath.format(set_type)


'test.txt'


(Pdb)  imgsetpath


'{:s}.txt'


...

'C:\\Users\\jjang\\Documents/github/ssd.pytorch/data/VOCdevkit/'


(Pdb)  os.path.join('data', 'VOC2007', 'ImageSets', 'Main', '{:s}.txt')


'{:s}.txt'


(Pdb)  set_type


'test'
